In [1]:
junction_id = {'Onnut': 'cluster_1088409501_272206263_5136790697_70702637',
     'PHRAKHANONG' : 'cluster_272206256_272448121_277360977',
#      'SUKHUMVIT71' : 'cluster_313173111_3810777044'
              }
number_phase = [3,3]
junction_name = list(junction_id.keys())

In [2]:
# import gym
# from gym import error, spaces
import gym
import lxml.etree as ET
import csv
import json
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch
import copy
import random
import pandas as pd
from multiprocessing.pool import ThreadPool
from itertools import zip_longest

In [3]:
namelane_csv = pd.read_csv('namelane_ONNUT.csv')
namelane_df = pd.DataFrame(namelane_csv, columns = ['name' , 'id'])
NAME = namelane_df.set_index('name')
ID = namelane_df.set_index('id')

In [4]:
NAME.loc[:,'id']

name
ONNUT_EB_0_0_LSXX                           -93727606#1_0
ONNUT_NB_0_0_LSXX                           751454884#3_0
ONNUT_NB_0_1_XSXX                           751454884#3_1
ONNUT_NB_0_2_XSXX                           751454884#3_2
ONNUT_NB_1_0_LSXX                           751454884#2_0
                                        ...              
SUKHUMVIT71_WB_3_1_XSXX    750035412#1-AddedOffRampEdge_1
SUKHUMVIT71_WB_3_2_XSXX    750035412#1-AddedOffRampEdge_2
SUKHUMVIT71_WB_4_0_XSXX                     750035412#1_0
SUKHUMVIT71_WB_4_1_XSXX                     750035412#1_1
SUKHUMVIT71_WB_4_2_XSXX                     750035412#1_2
Name: id, Length: 124, dtype: object

In [5]:
namedetector_csv = pd.read_csv('namedetector_ONNUT_extend.csv')
namedetector_df = pd.DataFrame(namedetector_csv, columns = ['name' , 'id'])
NAME_D = namedetector_df.set_index('name')
ID_D = namedetector_df.set_index('id')
listdetector = open("namedetector_ONNUT_extend.txt", "r")
detector = {}
for l in listdetector:
    l = l.strip().split(' ')
    if len(l)> 1:
        d = []
        for detec in l[2:]:
            if type(NAME_D.loc[detec,'id']) == str:
                d.append(NAME_D.loc[detec,'id'])
            else : d.append(NAME_D.loc[detec,'id'][0])
    detector[str(l[0])] = d
list_detector= list(detector.keys())

In [6]:
NAME_D.head()

,id
name,
ONNUT_SB_0_0_LSXX,D824116560#3_0
ONNUT_SB_0_1_XSXX,D824116560#3_1
ONNUT_SB_0_2_XSXX,D824116560#3_2
ONNUT_SB_0_3_XSXX,D824116560#3_3
ONNUT_SB_1_0_LSXX,D824116560#2_0


In [7]:
detector

{'ONNUT_SB_FP1_END': ['D824116560#3_0',
  'D824116560#3_1',
  'D824116560#3_2',
  'D824116560#3_3',
  'D824116560#2_0',
  'D824116560#2_1',
  'D824116560#2_2',
  'D824116560#2_3',
  'D824116560#1_0',
  'D824116560#1_1',
  'D824116560#1_2',
  'D824116560#1_3',
  'D824116560#0_0',
  'D824116560#0_1',
  'D824116560#0_2',
  'D824116560#0_3',
  'D824816455_0',
  'D824816455_1',
  'D824816455_2'],
 'ONNUT_WB_FP2_END': ['D156591171#2_0',
  'D156591171#2_1',
  'D156591171#2_2',
  'D156591171#0_0',
  'D156591171#0_1',
  'D156591171#0_2'],
 'ONNUT_WB_FP1_TP2': ['D-824456409#0_0',
  'D-824456409#0_1',
  'D-824456409#4_0',
  'D-824456409#4_1',
  'D-824456409#5_0',
  'D-824456409#5_1',
  'D-113135397#0_0',
  'D-113135397#0_1'],
 'PHRAKHANONG_EB_FP1_TP2': ['D336620983#5_0',
  'D336620983#5_1',
  'D336620983#5_2'],
 'SUKHUMVIT71_SB_FP2_TP3': ['D156591249#1_0',
  'D156591249#1_1',
  'D156591249#0.2206_0',
  'D156591249#0.2206_1'],
 'ONNUT_SB_FP2_TP3': ['D300825126#0_0',
  'D300825126#0_1',
  'D3008251

In [8]:
def get_mean_speed():
    
    speed = [traci.lanearea.getLastStepMeanSpeed(e) for e in NAME_D.loc[:,'id'] if traci.lanearea.getLastStepMeanSpeed(e)>=0]
#     print(speed)
    mean_speed = sum(speed)/max(len(speed),1)
#     print('speed',mean_speed)
    return mean_speed

In [9]:
def get_hot_encoding_current_phase():
    number_phase = [3,3]
    current_phase = [traci.trafficlight.getPhase(junction_id[key]) for key in junction_id.keys()]
#     current_phase = [0,2,1,1,1,1]
    hot_encoding_current_phase = np.array([])
    for i in range(len(current_phase)):
        binary_phase = np.zeros(number_phase[i])
        binary_phase[current_phase[i]] = 1
        hot_encoding_current_phase = np.concatenate((hot_encoding_current_phase, binary_phase), axis=None)
    return current_phase, hot_encoding_current_phase


In [10]:
# ID.loc['']#, '820373198#0', '820373196#0', '482209831#0']

In [11]:
def get_state_attention(current_phase, hot_encoding_current_phase):
#     current_phase = [ONNUT_PHASE]
    MAP_ONNUT = {0: ['ONNUT_WB_FP1_TP2', 'ONNUT_SB_FP1_END'],
                 1: ['ONNUT_SB_FP1_END', 'ONNUT_WB_FP1_TP2'],
                 2: ['ONNUT_WB_FP2_END'],
                }
    MAP = [MAP_ONNUT]
    state_attention = np.zeros(3)
    for i in range(1):
        for e in MAP[i][current_phase[i]]:
            occupancy = get_occupancy_average_percent(detector[e]) 
            Index_detector = list_detector.index(e) 
            state_attention[Index_detector] = occupancy
#     print(state_attention)
    state = np.concatenate((state_attention, hot_encoding_current_phase), axis=None)
    return state

In [12]:
def read_summary_xml(dateTimeObj):
    meanWaitingTime = []
    meanTravelTime = []
    meanSpeed = []
    tree = ET.parse('summary/summary3'+dateTimeObj+'.xml')
    summary = tree.getroot()
    for step in summary:
        list1 = step.attrib
        meanWaitingTime.append(float(list1["meanWaitingTime"]))
        meanTravelTime.append(float(list1["meanTravelTime"]))
        meanSpeed.append(float(list1["meanSpeed"]))
    meanWaitingTime_avg = sum(meanWaitingTime)/len(meanWaitingTime)
    meanTravelTime_avg = sum(meanTravelTime)/len(meanTravelTime)
    meanSpeed_avg = sum(meanSpeed)/len(meanSpeed)
    os.remove('summary/summary3'+dateTimeObj+'.xml')
    return meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg

In [13]:
#reset the environment
def start(dateTimeObj):
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "Onnut/onnut.sumocfg",
#                              "--summary-output", "summary/summary3"+dateTimeObj+".xml", 
                 '--start','true','--quit-on-end','true','--time-to-teleport','-1',
                '--lanechange.duration', '0.1'])

In [14]:
def get_occupancy_average_percent(detector_id): 
    #get occupancy average for all detector in list of detector_id and scale by (Vehicle Length + MinimumGap)/MinimumGap 
    #Vehicle Length = 4.62 MinimumGap = 2.37
    occupancy = (sum([traci.lanearea.getLastStepOccupancy(e)*traci.lanearea.getLength(e) for e in detector_id]))/\
                 sum([traci.lanearea.getLength(e) for e in detector_id])*((4.62+2.37)/4.62)
#     print(occupancy)
    return occupancy

In [15]:
def get_throughput():
    loopID = traci.inductionloop.getIDList()
    throughput = sum([traci.inductionloop.getLastStepVehicleNumber(i) for i in loopID if traci.inductionloop.getLastStepMeanSpeed(i) > 0])  #if traci.inductionloop.getLastStepMeanSpeed(i) > 0
    return throughput

In [16]:
def get_drawback():
#     laneID = traci.lane.getIDList()
    drawback = sum([traci.lanearea.getLastStepVehicleNumber(i) for i in NAME_D.loc[:,'id']])
    return drawback

In [17]:
def get_reward():
    throughput = 0
    for i in range(15):
        traci.simulationStep()
        throughput += get_throughput()
    
    drawback = get_drawback()
    reward = throughput - 0.04*drawback
    return reward, throughput, drawback

In [18]:
number_phase = [3]
def set_current_phase(action, current_phase):
    if action < 3:
        phase = action
        current_phase[0] = phase
    for i in range (1):
        traci.trafficlight.setPhase(junction_id[junction_name[i]], current_phase[i])
    return 

In [19]:
# ''.join(str(datetime.datetime.now()).split(' '))

In [20]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Step')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [21]:
class MyEnv3(gym.Env):
    def __init__(self, env_config):
        self.nearkill = 0
        self.episode = 0
        self.count = 0
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.drawbacks = 0
        self.current_phase = [1]
        self.done = False
        self.reward_memory = []
        self.log_action = []
        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.Box(low=-1, high=200, shape=(9,), dtype=np.float16)
        self.dateTimeObj = ''
        print(self.dateTimeObj)
        with open( "./Raytest/ray_results/onnut_occ.csv" , 'w', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writeheader()
        
    def reset(self):
        self.nearkill = 0
        self.episode += 1
        self.log_action = []
        self.reward_memory.append(self.rewards)
        self.count = 0
        dateTimeObj = datetime.datetime.now()
        self.dateTimeObj = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S-%f")
#         print(type(self.dateTimeObj))
        start(self.dateTimeObj)
        self.reward = 0
        self.rewards = 0
        self.throughputs = 0
        self.backlogs = 0
        print(self.reward_memory)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        state = get_state_attention(current_phase, hot_encoding_current_phase)
#         print(state)
        self.done = False
        self.current_phase = current_phase
        if len(self.reward_memory)%100 == 0:
            print('memory',self.reward_memory[-10:])
        return state 
    
    
    def step(self, action):
        set_current_phase(action, self.current_phase)
#         print(action)
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        self.reward, throughput, drawback= get_reward()
        self.rewards += self.reward
        self.throughputs += throughput
        self.drawbacks += drawback
        state = get_state_attention(current_phase, hot_encoding_current_phase)
        if np.isnan(self.reward) == True:
            print('HELP', type(self.reward))
        self.count += 1
        self.current_phase = current_phase
        self.log_action.append(str(action))
#         print('count', self.count)
        self.done = False
        mean_speed = get_mean_speed() 
        if mean_speed <= 2:
            self.nearkill += 1
        if mean_speed <= 2 and self.count >= 200 and self.nearkill > 10: #2880
            self.nearkill = 0
            traci.close()
            self.rewards -= 100000
            self.reward -= 100000
            self.done = True
            meanWaitingTime_avg = 0
            meanTravelTime_avg = 0
            meanSpeed_avg = 0
#             meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg = read_summary_xml(self.dateTimeObj)
            data_set = {"rewards": self.rewards, "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": mean_speed, "throughput": self.throughputs, "backlog": self.backlogs/2880, 
                        "action":self.log_action
                       }
            
            self.json_dump = json.dumps(data_set)
#             if self.episode%20 == 0:
#                 self.log_action_json = json.dumps(self.log_action)
#                 with open( "./Raytest/ray_results/action"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
#                     header = ['action']
#                     writer = csv.DictWriter(csv_file, fieldnames = header)
#                     writer.writerow({'action': self.log_action})
                                 
            with open("./Raytest/ray_results/onnut_occ.csv" , 'a', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writerow({'rewards': self.rewards, 
                                 'throughput': self.throughputs,
                                'backlog': self.backlogs/2880,
                                "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": mean_speed, "action":self.log_action})
        if self.count >= 960: #2880
            traci.close()
            self.done = True
            meanWaitingTime_avg = 0
            meanTravelTime_avg = 0
            meanSpeed_avg = 0
#             meanWaitingTime_avg,meanTravelTime_avg,meanSpeed_avg = read_summary_xml(self.dateTimeObj)
            data_set = {"rewards": self.rewards, "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": mean_speed, "throughput": self.throughputs, "backlog": self.backlogs/2880, 
                        "action":self.log_action
                       }
            
            self.json_dump = json.dumps(data_set)
#             if self.episode%20 == 0:
#                 self.log_action_json = json.dumps(self.log_action)
#                 with open( "./Raytest/ray_results/action"+self.dateTimeObj+".csv" , 'a', newline='') as csv_file:
#                     header = ['action']
#                     writer = csv.DictWriter(csv_file, fieldnames = header)
#                     writer.writerow({'action': self.log_action})
                                 
            with open( "./Raytest/ray_results/onnut_occ.csv" , 'a', newline='') as csv_file:
                header = ['rewards', 'throughput','backlog',"meanWaitingTime", "meanTravelTime","meanSpeed","action"]
                writer = csv.DictWriter(csv_file, fieldnames = header)
                writer.writerow({'rewards': self.rewards, 
                                 'throughput': self.throughputs,
                                'backlog': self.backlogs/2880,
                                "meanWaitingTime": meanWaitingTime_avg, "meanTravelTime": meanTravelTime_avg,
                       "meanSpeed": meanSpeed_avg, "action":self.log_action})

        
        return_state = np.array(state).astype(np.float16)
#         print(return_state)
        info = {"throughput": throughput,
                "drawback":drawback
                }
        info = {**info}
#         print(info)
        return np.int8(return_state) , self.reward, self.done, info

In [22]:
police_action = [[1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [1, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [2, None], [1, None], [1, None], [1, None], [1, None]]
action_list = [e[0] for e in police_action]

In [23]:
# action_list = ['2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '1', '0', '1', '0', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '0', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '2', '1', '2', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '2', '2', '1', '2', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '0', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '0', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '0', '1', '0', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '0', '2', '1', '0', '2', '1', '0', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2']

In [24]:
# action_list

In [25]:
action_list = ['1', '1', '2', '2', '2', '2', '1', '2', '0', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '1', '1', '0', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '0', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '0', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '0', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '2', '1', '0', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '0', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '1', '1', '1', '2', '0', '1', '2', '1', '1', '2', '1', '1', '2', '1', '2', '1', '2', '1', '2', '0']

In [26]:
current_phase = [1]
rewards = 0
throughputs = 0
backlogs = 0
mean_speed = 0
start("rrrr")
action = [int(a) for a in action_list]
for i in range (len(action)): 
    set_current_phase(action[i], current_phase)
    current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
    reward, throughput, backlog= get_reward()
    rewards += reward
    throughputs += throughput
    backlogs += backlog
    mean_speed += get_mean_speed()
    state = get_state_attention(current_phase, hot_encoding_current_phase)
traci.close()

In [27]:
# traci.close()

In [28]:
rewards

-7086.639999999994

In [29]:
throughputs

0

In [30]:
backlogs/960

184.54791666666668

In [31]:
mean_speed/2880

2.138783209446438